# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
admin_predict = pd.read_csv("./Admission_Predict.csv")

Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
admin_predict.columns = admin_predict.columns.str.strip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
admin_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
admin_predict.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
admin_predict.set_index('Serial No.', drop=False, inplace=True)
admin_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here
admin_predict['GRE_CGPA_comb'] = admin_predict['GRE Score'].astype('string') + '_' + admin_predict['CGPA'].astype('string')
display(admin_predict['GRE_CGPA_comb'].unique().shape)
display(admin_predict.shape)
# -> same number of rows for serial number and unique combinations of GRE Score and CGPA -> q.e.d.

(385,)

(385, 10)

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [8]:
# your code here
admin_predict.set_index(['GRE Score', 'CGPA'], drop = True, inplace = True)
admin_predict

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb
GRE Score,CGPA,,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65
316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0
322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67
314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21
330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34
...,...,...,...,...,...,...,...,...,...
324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04
325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11
330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [9]:
# your code here
admin_predict.reset_index(drop=False, inplace=True)
admin_predict

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34
...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [10]:
# your code here
admin_predict[(admin_predict['CGPA'] > 9) & (admin_predict['Research'] == 1)]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34
10,328,9.10,11,112,4,4.0,4.5,1,0.78,328_9.1
19,328,9.50,20,116,5,5.0,5.0,1,0.94,328_9.5
20,334,9.70,21,119,5,5.0,4.5,1,0.95,334_9.7
...,...,...,...,...,...,...,...,...,...,...
379,329,9.23,380,111,4,4.5,4.0,1,0.89,329_9.23
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# your code here
admin_predict[(admin_predict['CGPA'] > 9) & (admin_predict['SOP'] < 3.5)]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb
28,338,9.40,29,118,4,3.0,4.5,1,0.91,338_9.4
62,327,9.02,63,114,3,3.0,3.0,0,0.61,327_9.02
140,326,9.11,141,114,3,3.0,3.0,1,0.83,326_9.11
217,324,9.01,218,111,4,3.0,3.0,1,0.82,324_9.01
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col, col_std, col_mean):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    # your code here
    # How to pass a column name to function ???
    # std_col = admin_predict[col].std()
    # mean_col = admin_predict[col].values.mean()
    return (col - col_std) / col_mean

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [13]:
# your code here
admin_predict['CGPA_std'] = admin_predict['CGPA'].apply(standardize, args=(admin_predict['CGPA'].std(), admin_predict['CGPA'].mean()))
admin_predict['GRE_std'] = admin_predict['GRE Score'].apply(standardize, args=(admin_predict['GRE Score'].std(), admin_predict['GRE Score'].mean()))
admin_predict['LOR_std'] = admin_predict['LOR'].apply(standardize, args=(admin_predict['LOR'].std(), admin_predict['LOR'].mean()))
admin_predict

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65,1.052404,1.027495,1.050098
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0,0.860479,0.961193,0.758652
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67,0.938412,0.980136,0.467206
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21,0.884906,0.954878,0.612929
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34,1.016346,1.005394,0.612929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04,0.981450,0.986450,0.758652
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11,0.989592,0.989608,0.758652
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45,1.029141,1.005394,1.050098
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78,0.951207,0.948564,0.904375


We will generate the decision choice at random using the code below. Please run the cell.

In [14]:
# Libraries
from random import choices

In [15]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=admin_predict.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [16]:
# your code here
admin_predict['deciding_column'] = admin_predict.lookup(admin_predict.index, decision_choice)
decision_choice 
admin_predict

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65,1.052404,1.027495,1.050098,1.050098
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0,0.860479,0.961193,0.758652,0.860479
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67,0.938412,0.980136,0.467206,0.938412
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21,0.884906,0.954878,0.612929,0.884906
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34,1.016346,1.005394,0.612929,1.016346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04,0.981450,0.986450,0.758652,0.981450
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11,0.989592,0.989608,0.758652,0.989608
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45,1.029141,1.005394,1.050098,1.050098
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78,0.951207,0.948564,0.904375,0.948564


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [17]:
# your code here
admin_predict['decision'] = np.where(admin_predict['deciding_column'] > 0.8, 1, 0)
admin_predict

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE_CGPA_comb,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65,1.052404,1.027495,1.050098,1.050098,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0,0.860479,0.961193,0.758652,0.860479,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67,0.938412,0.980136,0.467206,0.938412,1
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21,0.884906,0.954878,0.612929,0.884906,1
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34,1.016346,1.005394,0.612929,1.016346,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04,0.981450,0.986450,0.758652,0.981450,1
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11,0.989592,0.989608,0.758652,0.989608,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45,1.029141,1.005394,1.050098,1.050098,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78,0.951207,0.948564,0.904375,0.948564,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [18]:
# your code here
admin_predict['decision'].sum()

301

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [19]:
# your code here
admin_predict.columns = admin_predict.columns.str.lower().str.replace('.', '').str.replace(' ', '_')
admin_predict

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,gre_cgpa_comb,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65,1.052404,1.027495,1.050098,1.050098,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0,0.860479,0.961193,0.758652,0.860479,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67,0.938412,0.980136,0.467206,0.938412,1
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21,0.884906,0.954878,0.612929,0.884906,1
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34,1.016346,1.005394,0.612929,1.016346,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04,0.981450,0.986450,0.758652,0.981450,1
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11,0.989592,0.989608,0.758652,0.989608,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45,1.029141,1.005394,1.050098,1.050098,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78,0.951207,0.948564,0.904375,0.948564,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [20]:
# your code here
admin_predict['adjusted_gre'] = np.where(admin_predict['university_rating'] >= 4, admin_predict['gre_score'] + 10 , admin_predict['gre_score'])
labels = ['low', 'medium', 'high', 'very high']
bins = pd.DataFrame(pd.cut(admin_predict['adjusted_gre'], 4, labels=labels))
bins.columns = ['adjusted_gre_bin']
admin_predict = admin_predict.merge(bins, left_index=True, right_index=True)
admin_predict

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,gre_cgpa_comb,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre,adjusted_gre_bin
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337_9.65,1.052404,1.027495,1.050098,1.050098,1,347,very high
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316_8.0,0.860479,0.961193,0.758652,0.860479,1,316,medium
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322_8.67,0.938412,0.980136,0.467206,0.938412,1,322,high
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314_8.21,0.884906,0.954878,0.612929,0.884906,1,314,medium
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330_9.34,1.016346,1.005394,0.612929,1.016346,1,340,very high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,324_9.04,0.981450,0.986450,0.758652,0.981450,1,324,high
381,325,9.11,382,107,3,3.0,3.5,1,0.84,325_9.11,0.989592,0.989608,0.758652,0.989608,1,325,high
382,330,9.45,383,116,4,5.0,4.5,1,0.91,330_9.45,1.029141,1.005394,1.050098,1.050098,1,340,very high
383,312,8.78,384,103,3,3.5,4.0,0,0.67,312_8.78,0.951207,0.948564,0.904375,0.948564,1,312,medium
